# **Apple Health Data XML -> CSV**
---

### Import Libraries

In [1]:
import xml.etree.ElementTree as ET 
import pandas as pd 
import numpy as np
import datetime as dt 

# **Apple Health Data Export: Data Exports to `XML`**
---

### Parse XML File into Element Tree Object 

In [2]:
# Parse XML file into an Element Tree Object
tree = ET.parse('data/apple_health_export/export.xml')

#extract attributes for every health record 
root = tree.getroot()
record_list = [x.attrib for x in root.iter('Record')]

### Create Pandas Dataframe 

In [3]:
# Create Pandas Dataframe 
record_data = pd.DataFrame(record_list)

# Proper Type to Localized Timezone Dates 
for col in ['creationDate', 'startDate', 'endDate']:
    record_data[col] = pd.to_datetime(record_data[col]).dt.tz_localize(None)

# Value is Numeric, Nan if Fails 
record_data['value'] = pd.to_numeric(record_data['value'], errors='coerce')

# some records do NOT measure anything (just count occurences)
# fill with 1.0 (one time) makes easier to aggregate
record_data['value'] = record_data['value'].fillna(1.0)

# shorter observation names
record_data['type'] = record_data['type'].str.replace('HKQuantityTypeIdentifier', '')
record_data['type'] = record_data['type'].str.replace('HKCategoryTypeIdentifier', '')
record_data.tail()

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device
177481,HeadphoneAudioExposureEvent,Brighton Mini,15.6,NaN,2022-11-22 16:41:27,2022-11-16 18:17:49,2022-11-22 16:41:27,1.0,NaN
177482,HeadphoneAudioExposureEvent,Brighton Mini,15.6,NaN,2022-11-28 14:31:18,2022-11-22 16:41:27,2022-11-28 14:31:18,1.0,NaN
177483,HeadphoneAudioExposureEvent,Brighton Mini,15.6,NaN,2022-11-29 16:22:42,2022-11-28 14:31:18,2022-11-29 16:22:42,1.0,NaN
177484,HeadphoneAudioExposureEvent,Brighton Mini,15.6,NaN,2022-12-02 08:18:16,2022-11-29 16:22:42,2022-12-02 08:18:15,1.0,NaN
177485,HeadphoneAudioExposureEvent,Brighton Mini,15.6,NaN,2022-12-06 09:22:06,2022-12-02 08:18:15,2022-12-06 09:22:06,1.0,NaN


In [4]:
record_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177486 entries, 0 to 177485
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   type           177486 non-null  object        
 1   sourceName     177486 non-null  object        
 2   sourceVersion  177486 non-null  object        
 3   unit           177356 non-null  object        
 4   creationDate   177486 non-null  datetime64[ns]
 5   startDate      177486 non-null  datetime64[ns]
 6   endDate        177486 non-null  datetime64[ns]
 7   value          177486 non-null  float64       
 8   device         177354 non-null  object        
dtypes: datetime64[ns](3), float64(1), object(5)
memory usage: 12.2+ MB


# **Usable Dataframes From `XML` Data**
---

# Pull Daily `StepCount` With *Intraday Times*


In [5]:
# Make Dataframe with Only Step Count Data (Keeping Intraday Data)
step_count_df = record_data[record_data['type'] == 'StepCount']


# Only Keep StartDate
step_count_df = step_count_df.drop(['type','sourceName', 'sourceVersion', 'unit', 'creationDate', 'endDate', 'device'], axis=1)

# Split StartDate into Date and Time
step_count_df['date'] = pd.to_datetime(step_count_df['startDate']).dt.date
step_count_df['time'] = pd.to_datetime(step_count_df['startDate']).dt.hour
step_count_df = step_count_df.drop(['startDate'], axis=1)

# need this because without you end up with Unnamed:0 column from csv 
step_count_df.set_index('date', inplace=True)

step_count_df

,value,time
date,,
2021-12-21,220.0,13
2021-12-21,421.0,13
2021-12-21,119.0,13
2021-12-21,690.0,14
2021-12-21,201.0,14
...,...,...
2022-12-07,2.0,10
2022-12-07,53.0,10
2022-12-07,568.0,12


### Pandas DF to CSV

In [6]:
step_count_df.to_csv('data/step_count_intraday.csv')

# Pull `StepCount` and `DistanceWalkingRunning` to Dataframe - Daily Sums

In [7]:
# pivot dataframe 
pivot_df = record_data.pivot_table(index='endDate', columns='type', values='value')
# sum StepCount and DistanceWalkingRunning 
df = pivot_df.resample('D').agg({'StepCount' : sum,
                                 'DistanceWalkingRunning' : sum})
                                 
df.index = df.index.strftime('%Y-%m-%d')
df

type,StepCount,DistanceWalkingRunning
endDate,,
2021-09-10,3246.0,1.309123
2021-09-11,10003.0,4.154720
2021-09-12,4812.0,2.097997
2021-09-13,4672.0,1.776384
2021-09-14,5914.0,2.324539
...,...,...
2022-12-03,12476.0,4.832174
2022-12-04,3428.0,1.365090
2022-12-05,16428.0,6.891993


### Pandas DF to CSV 

In [8]:
df.to_csv('data/steps_distance.csv')